In [1]:
# Imports
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the data
players_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/players.csv')
plays_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/full_play_3.csv')

In [3]:
# Rename the defender feature to displayName
plays_df.rename(columns = {'defender': 'displayName'}, inplace = True)

In [4]:
# Merge the plays and games dataframes
full_play_df = pd.merge(plays_df, players_df, on = 'displayName', how = 'inner')

In [5]:
# Clean up the columns
full_play_df = full_play_df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], axis=1)

In [6]:
# The number of plays for each position and result
count_df = full_play_df.groupby(['position', 'passResult'])['playId'].count()
count_df = count_df.reset_index()

In [7]:
# Pivot the counts
pivot_df = count_df.pivot(index='position', columns='passResult', values='playId')

In [8]:
# Reset the index
pivot_df.reset_index()

passResult,position,C,I,IN
0,CB,4957.0,3080.0,158.0
1,DB,261.0,131.0,12.0
2,DE,62.0,17.0,2.0
3,DT,6.0,3.0,NaN
4,FB,1.0,NaN,NaN
5,FS,1078.0,722.0,83.0
6,ILB,1111.0,419.0,24.0
7,LB,733.0,241.0,14.0
8,MLB,526.0,180.0,15.0
9,OLB,1347.0,518.0,31.0


In [9]:
# Calculate the completion percentage
pivot_df['compPercent'] = pivot_df['C'] / (pivot_df['C'] + pivot_df['I'] + pivot_df['IN'])

In [10]:
pivot_df.to_excel('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/final/matchup1.xlsx', index=True)

In [11]:
# The number of plays for each scheme and position
count_df = full_play_df.groupby(['scheme', 'position'])['playId'].count()
count_df = count_df.reset_index()

In [12]:
# Drop schemes equal to None
filtered_df = count_df[count_df['scheme'] != 'None']

In [13]:
# Reclassify all linebacker positions to LB
filtered_df.loc[(filtered_df['position'] == 'ILB') | (filtered_df['position'] == 'MLB') | 
                (filtered_df['position'] == 'OLB'), 'position'] = 'LB'

In [14]:
# Reclassify all defensive backs to DB
filtered_df.loc[(filtered_df['position'] != 'LB'), 'position'] = 'DB'

In [15]:
# The number of plays for each scheme and position
final_df = filtered_df.groupby(['scheme', 'position'])['playId'].sum()
final_df = final_df.reset_index()

In [16]:
# Pivot the counts
fpivot_df = final_df.pivot(index='scheme', columns='position', values='playId')

In [17]:
# Calculate the completion percentage
fpivot_df['posPercent'] = fpivot_df['LB'] / (fpivot_df['LB'] + fpivot_df['DB'])

In [18]:
# Reset the index
fpivot_df.reset_index()

position,scheme,DB,LB,posPercent
0,cover-0,1033,300,0.225056
1,cover-1,1235,368,0.229570
2,cover-2,709,292,0.291708
3,cover-3,5522,2148,0.280052
4,cover-4,3419,1695,0.331443


In [19]:
fpivot_df.to_excel('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/final/matchup2.xlsx', index=True)